# Traitement de données audio en Python
## Séance 2 : Manipulations simples avec le package [Pydub](http://pydub.com/)

Nous allons présenter des exemples de traitement sur le fichier [electric_cello.wav](../audio/electric_cello.wav)

Évidemment pour pouvoir utiliser `pydub` il faut l’avoir installé au préalable.

`python3 -m pip install pydub` ou `conda install pydub`

Comme vous l’avez vu, un fichier son est constitué d’un ensemble d’échantillons. Pour un fichier wav PCM on aura une suite d’octets composée d’un en-tête suivi pour les échantillons.

Les classes et les fonctions de `pydub` vont vous permettre de manipuler simplement les données contenues dans un fichier son. Dans différents formats (wav, mp3, ogg, …) mais pour ce TP nous nous en tiendrons au format wav PCM.

## Exercices de la séance 1

En utilisant le package Pydub et sa documentation :

1. Afficher les paramètres d’encodage de notre fichier
2. Extraire les 10 premières secondes de notre fichier son et les écouter
3. Extraire les 20 dernières secondes et les exporter en wav

### 1. Afficher les paramètres d’encodage de notre fichier

Il faut commencer par lire la documentation de la classe [AudioSegment](https://github.com/jiaaro/pydub/blob/master/API.markdown#audiosegment) et repérer comment accéder aux informations d’encodage.

In [1]:
# Afficher les paramètres d’encodage de notre fichier

from pydub import AudioSegment

song = AudioSegment.from_wav("../audio/electric_cello.wav")
print(f"nombre de canaux : {song.channels}, \
\nfréquence d’échantillonage : {song.frame_rate}, \
\nrésolution de chaque échantillon : {song.sample_width}, \
\nnombre de frames : {song.frame_count()}")

nombre de canaux : 2, 
fréquence d’échantillonage : 48000, 
résolution de chaque échantillon : 2, 
nombre de frames : 1170432.0


### 2. Extraire les 10 premières secondes et les écouter

`song` est un objet de la classe `AudioSegment`. Au vu de la [documentation](http://pydub.com/) et du [code source](https://github.com/jiaaro/pydub/blob/master/pydub/audio_segment.py) on comprend que `song` est un [*iterable*](https://docs.python.org/3/glossary.html#term-iterable) et une [*sequence*](https://docs.python.org/3/glossary.html#term-sequence). On peut utiliser un [*slice*](https://docs.python.org/3/glossary.html#term-slice) pour le découper.

In [2]:
# Extraire les 10 premières secondes et les écouter

from pydub import AudioSegment
from pydub.playback import play

song = AudioSegment.from_wav("../audio/electric_cello.wav")
first_10_seconds = song[0:10000] # extraction des 10 premières secondes à l’aide d’un slice
print(len(first_10_seconds))
#play(first_10_seconds)

10000


### 3. Extraire les 20 dernières secondes et les exporter en wav

Là aussi la documentation nous donne les réponses ou presque. Encore une fois on découpe la suite d’échantillons et `pydub` nous permet de générer un fichier wav avec notre morceau découpé.
La fonction `export` ajoute l’en-tête wav avec les infos nécessaires à notre découpe puis crée un fichier à l’emplacement désiré. 

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

song = AudioSegment.from_wav("../audio/electric_cello.wav")
last_20_seconds = song[-20000:] # extraction des 10 premières secondes à l’aide d’un slice
last_20_seconds.export("../audio/electric_cello_last20s.wav", format="wav")

## Exploration de `pydub`

Pydub est un package de haut niveau qui a l’avantage d’être simple à utiliser.  
Mais comme le dit la [documentation](https://github.com/jiaaro/pydub/blob/master/API.markdown#api-documentation), une partie des fonctions ne sont pas signalées dans la doc. Il faut donc aussi explorer le code source pour apprendre à utiliser toutes les fonctions.

### 1. Génération de silence

Avec la fonction `silence` générer 5 secondes de silence

In [ ]:
silence = AudioSegment.silent(duration=5000, frame_rate=48000)
len(silence)
#play(silence)

### 2. Concaténation

Ajouter ce silence avant les 10 premières secondes

In [ ]:
silence_10 = silence + first_10_seconds
len(silence_10)
#play(silence_10)

La concaténation est réalisée grâce à l’opérateur `+` qui est déjà utilisé pour concaténer des chaînes de caractères (`str`, voir doc [ici](https://realpython.com/python-string-split-concatenate-join/#concatenating-and-joining-strings) par exemple).  
Pour que la concaténation avec `+` fonctionne, pydub surcharge la fonction `__add__` (voir le code source [ici](https://github.com/jiaaro/pydub/blob/996cec42e9621701edb83354232b2c0ca0121560/pydub/audio_segment.py#L362))

#### 3. Augmentation et diminution du volume

Répéter les 2 premières secondes 3 fois : à volume normal, -6db, +6db

In [ ]:
# extraire les 2 premières secondes dans une variable two_seconds
two_seconds = song[0:2000]
# diminuer de 6 db et stocker dans variable two_seconds_6minus
two_seconds_6minus = two_seconds - 6
# augmenter de 6 db et stocker dans variable two_seconds_6plus
two_seconds_6plus = two_seconds + 6
# concaténer two_seconds, two_seconds_6minus et two_seconds_6plus
two_seconds_3fois = two_seconds + two_seconds_6minus + two_seconds_6plus
len(two_seconds_3fois)
play(two_seconds_3fois)

Ici aussi pydub ajoute un comportement à l’opérateur `+`. En fonction du type des opérandes, `+` appliquera une concaténation ou un gain de volume.

### 4. *Fade*

Voir les fonctions `fade`, `fade_in`, `fade_out`

In [ ]:
# exemple pour un fade_in de 2,5 secondes sur les 5 premières secondes

play(song[0:5000].fade_in(duration=2500))

### 5. À l’envers

Comment faire pour lire notre enregistrement à l’envers ? À partir de la fin jusqu’au début.

In [ ]:
play(song.reverse())

Une autre façon, en manipulant le tableau de données

In [ ]:
data = song.get_array_of_samples()
reverse = song._spawn(data[::-1])
play(reverse)

5.1 Avec notre enregistrement à l’envers nous allons créer un nouvel objet `AudioSegment ` de 2 canaux avec dans le canal gauche l’enregistrement à l’endroit et dans le canal droit l’enregistrement à l’envers 

In [3]:
# on doit générer deux objets AudioSegment sur un seul canal
# pour cela on utilise la fonction `split_to_mono` qui renvoie une liste
# (si deux canaux alors deux éléments dans la liste)
# on ne conserve que le premier élément de la liste
left_channel = song.split_to_mono()[0]

# idem pour le morceau inversé
right_channel = song.reverse().split_to_mono()[0]

# `from_mono_audiosegments` permet de construire un segment stéréo à partir de deux segments mono
stereo_sound = AudioSegment.from_mono_audiosegments(left_channel, right_channel)
play(stereo_sound)


Input #0, wav, from '/tmp/tmpk_oxstic.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:24.38, bitrate: 1536 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 48000 Hz, 2 channels, s16, 1536 kb/s


### 6. Détection de silence

Ici aussi on va utiliser une fonction existante : `detect_silence` (voir le code source [ici](https://github.com/jiaaro/pydub/blob/996cec42e9621701edb83354232b2c0ca0121560/pydub/silence.py#L9))  
Cette fonction fait partie du module `silence` qu’il faudra ajouter à nos imports.

C’est une fonction de haut niveau, son exécution pourra être un peu plus lente que les fonctions vus précedemment.

In [ ]:

from pydub.silence import detect_silence

# a minima vous devez appeler detect_silence avec deux arguments
# le segment à analyser et le seuil de détection en différence de db
# essayer plusieurs seuils pour arriver à un résultat satisfaisant
silences = detect_silence(song, silence_thresh=-20)

# le résultat est une liste de segments de silences
# type(silences)
first_silence = silences[0]
print(f"le premier silence dure {len(first_silence)} secondes")
print(f"début : {first_silence[0]} ms")
print(f"fin : {first_silence[1]} ms")

### 7. *Overlay*

L’*overlay* permet d’ajouter un segment audio sur un segment audio, comme une surcouche.  

Vous pouvez utiliser la fonction `overlay`, elle est commentée [ici](https://github.com/jiaaro/pydub/blob/master/API.markdown#audiosegmentoverlay)

In [4]:
# Ici on ajoute une surcouche avec les 2 premières secondes du morceau
# l’overlay commence à 2 secondes et est répété en boucle
overlayed = song.overlay(song[0:2000], position=2000, loop=True)
play(overlayed)

Input #0, wav, from '/tmp/tmp01c2w7ai.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:24.38, bitrate: 1536 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 48000 Hz, 2 channels, s16, 1536 kb/s


### 8. Quel algorithme pour une fonction de *delay* ?

Le *delay* est un effet très utilisé dans la musique populaire et la musique concrète. Par exemple dans [Run Like Hell](https://youtu.be/j2s8yGMEbSs?si=-lKBfNnR7vuAI432) des Pink Floyd, plein de titres de U2 ([Pride](https://youtu.be/LHcP4MWABGY?si=7cB4eGOVhGB80nR9&t=14) par exemple) ou encore dans le dub ([exemple](https://youtu.be/vQXoxh0wwKc?si=yRmN8t8ljeGG7Xbb) avec un titre de Groundation).

C’est un effet d’écho où un extrait du signal est répété plusieurs fois.